In [2]:
import pandas as pd
import numpy as np
import urllib
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize

In [3]:
from gensim.models import word2vec
from sklearn.manifold import TSNE
import plotly_express as px

unable to import 'smart_open.gcs', disabling that module


In [18]:
data = pd.read_csv('/home/yk4we/ETA_final/newzy.csv', sep ='|')
data.head()

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url
0,39,Google News,"50 Years on, Finding Profit in 'Truth' on JFK ...","Washington Post50 Years on, Finding Profit in ...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
1,38,Google News,"Nearly 10 years later, families and officials ...","Washington PostNearly 10 years later, families...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
2,37,Google News,"Cuccinelli, running hard, has been hobbled in ...","Washington PostCuccinelli, running hard, has b...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
3,36,Google News,Chuck Schumer endorses Hillary Clinton for Pre...,New York Daily NewsChuck Schumer endorses Hill...,11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
4,35,Google News,Sec. Kerry issues apparent warning to Iran - C...,Ahram OnlineSec. Kerry issues apparent warning...,11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...


In [19]:
data.doc_source.unique()

array(['Google News', 'PowerLine', 'Reuters', 'Guardian', 'Daily Kos',
       'Drudge Report', 'Real Clear Politics', 'NPR', 'Fox',
       'New York Times', 'Breitbart', 'CNN', 'US News', 'UPI Latest',
       'Politico Magazine', nan], dtype=object)

In [20]:
data.loc[data.doc_source == 'Daily Kos']

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url
70,96,Daily Kos,Obama has done nothing to address income inequ...,Income and wealth inequality is a festering wo...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
71,97,Daily Kos,"Anti-racism is not 'anti-white,' no matter wha...",The virulent meme spread across our nation wit...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
72,98,Daily Kos,Abbreviated Pundit Round-up: Angry does no...,Chrystia Freeland sees a fight that's not not ...,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
73,99,Daily Kos,Sunday Talk: Haters gonna hate,"No doubt, the bestest thing about America is o...",11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
74,100,Daily Kos,Gore on fire: Says humans treating atmosphere ...,About time Al Gore has been everywhere lately....,11/03/2013,http://feeds.dailykos.com/~r/dailykos/index/~3...
...,...,...,...,...,...,...
922187,922223,Daily Kos,CNN's new Republican hack is ratioed trying to...,"Earlier today, former Republican Rep. from Wis...",10/31/2019,http://feeds.dailykosmedia.com/~r/dailykos/ind...
922188,922224,Daily Kos,California announces investigation into PG&amp...,"On Monday, the California Public Utilities Com...",10/31/2019,http://feeds.dailykosmedia.com/~r/dailykos/ind...
922189,922225,Daily Kos,Chicago teachers say 0.5% of the schools budge...,Chicago teachers say that just half of one per...,10/31/2019,http://feeds.dailykosmedia.com/~r/dailykos/ind...
922190,922226,Daily Kos,How do we engage people who've been sitting on...,"Ever wondered how, precisely, social movements...",10/31/2019,http://feeds.dailykosmedia.com/~r/dailykos/ind...


In [104]:
#data.loc[data.doc_source == 'New York Times']

In [131]:
# nyt = pd.read_csv('/home/yk4we/ETA_final/nyt.csv')
# len(nyt)

155

In [21]:
data = data.loc[(data['doc_source'] == 'Daily Kos') | (data['doc_source'] == 'PowerLine')]
data = data.reset_index(drop = True)
data.sample(5)

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url
10132,174205,Daily Kos,Cartoon: The Choice is Yours,Support Fake Comix!! www.patreon.com/keefknight,06/13/2017,http://feeds.dailykos.com/~r/dailykos/index/~3...
14058,269648,Daily Kos,"Mitch McConnell had a very, very bad Tuesday","Some days, even in America in 2017, there's ju...",09/27/2017,http://feeds.dailykos.com/~r/dailykos/index/~3...
21299,949197,PowerLine,Lee Smith: CNN & the Steele Dossier,(Scott Johnson) Lee Smith is the author of The...,11/29/2019,http://feeds.powerlineblog.com/~r/powerlineblo...
12564,231646,Daily Kos,Colorado Republican feels the heat back home f...,Sen. Cory Gardner (R-CO) probably felt he was ...,08/16/2017,http://feeds.dailykos.com/~r/dailykos/index/~3...
5169,63100,Daily Kos,Utah congressman bolts an hour early from cons...,Rep. Jason Chaffetz (UT-03) held a town hall t...,02/10/2017,http://feeds.dailykos.com/~r/dailykos/index/~3...


In [24]:
#create author column
# data['author'] = data.loc[data.doc_source =='PowerLine'].doc_content.apply(lambda x: str(x).split(')')[0].strip('('))

In [22]:
#remove author name in content in Powerline
data.doc_content = data.loc[data.doc_source == 'PowerLine'].doc_content.apply(lambda x: str(x).strip(str(x).split(')')[0]).strip(')'))
data

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url
0,48,PowerLine,These Zins Are a Dusi,I’ve been falling behind on many regular feat...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...
1,49,PowerLine,NY Times Editorial Board Hacked by The Onion,"At least I think it was the Onion. Someone, a...",11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...
2,50,PowerLine,The line Obama crossed,Obamacare was enacted on a foundational set o...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...
3,51,PowerLine,Sham/Wow,It’s unfair to compare and contrast Barack Ob...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...
4,52,PowerLine,A Quarterback Controversy With a Political Twist?,Public Policy Polling ran a survey here in Mi...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...
...,...,...,...,...,...,...
22035,1026329,PowerLine,The Democrats’ Bernie Dilemma: He Exposes Thei...,"I think it was Tom Bethell, back when he wrot...",02/27/2020,http://feeds.powerlineblog.com/~r/powerlineblo...
22036,1026330,PowerLine,Trump DOJ weighs in against Harvard’s discrimi...,The Trump Department of Justice has steadfast...,02/27/2020,http://feeds.powerlineblog.com/~r/powerlineblo...
22037,1026331,PowerLine,Trump Steps Out on Coronavirus,No one in public life has forgotten what the ...,02/27/2020,http://feeds.powerlineblog.com/~r/powerlineblo...
22038,1026332,PowerLine,A Legal Blow to “Sanctuary” Jurisdictions,Today the 2nd Circuit Court of Appeals ruled ...,02/27/2020,http://feeds.powerlineblog.com/~r/powerlineblo...


In [26]:
# create news_id
# data['news_id'] = data['doc_source'].apply(lambda x: 1 if x == 'PowerLine' else 2)
# data.sample(10)

In [23]:
# create year column
data['year'] = data.doc_date.apply(lambda x: x.split('/')[2])
data.sample(5)

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url,year
16802,466874,PowerLine,Confirm Jeff Clark,"For conservatives, the rolling back of oppres...",05/09/2018,http://feeds.powerlineblog.com/~r/powerlineblo...,2018
1592,16065,PowerLine,"The Stretch Drive (5), Thiel Edition",The news is going to come fast and furious th...,10/31/2016,http://feeds.powerlineblog.com/~r/powerlineblo...,2016
13471,254768,PowerLine,Irma update,Our Florida hurricane correspondent provides ...,09/11/2017,http://feeds.powerlineblog.com/~r/powerlineblo...,2017
4337,51097,Daily Kos,"Abbreviated Pundit Round-up: Donald Trump, Cha...",NaN,01/23/2017,http://feeds.dailykos.com/~r/dailykos/index/~3...,2017
6777,99603,Daily Kos,Morning Digest: The one-and-only Sharron Angle...,NaN,03/22/2017,http://feeds.dailykos.com/~r/dailykos/index/~3...,2017


In [24]:
# LIB table
LIB = data[['doc_id','doc_title','year','doc_source']].set_index('doc_id')
LIB.head()

,doc_title,year,doc_source
doc_id,,,
48,These Zins Are a Dusi,2013,PowerLine
49,NY Times Editorial Board Hacked by The Onion,2013,PowerLine
50,The line Obama crossed,2013,PowerLine
51,Sham/Wow,2013,PowerLine
52,A Quarterback Controversy With a Political Twist?,2013,PowerLine


## Tokenization

#### DOCtable

In [25]:
#segmented into sentences 
nltk.download('punkt')
df = data.doc_content.apply(lambda x: pd.Series(nltk.sent_tokenize(str(x))))
df = df.set_index(data.doc_id).stack().to_frame().rename(columns={0:'sent_str'})
df

[nltk_data] Downloading package punkt to /home/yk4we/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


sent_str
doc_id                                                       
48      0    I’ve been falling behind on many regular feat...
        1                   So here’s a quick catch up guide.
        2   Old timers of Paso Robles wines all know the D...
        3   The Dusi family’s Zinfandel vineyards go back ...
        4   For decades, if you wanted to find the best Zi...
...                                                       ...
1026333 13  They have repeatedly broken into chants of “Tr...
        14  I took this picture in the East Room last Fall...
        15      At least 95% of those in the room were black.
        16  They loved Trump, and Trump loved them:  Casua...
        17  But alleging that the President of the United ...

[243742 rows x 1 columns]

In [26]:
# DOC table
DOC = df.copy()
DOC.head()

sent_str
doc_id                                                     
48     0   I’ve been falling behind on many regular feat...
       1                  So here’s a quick catch up guide.
       2  Old timers of Paso Robles wines all know the D...
       3  The Dusi family’s Zinfandel vineyards go back ...
       4  For decades, if you wanted to find the best Zi...

In [13]:
#DOC.to_csv('/home/yk4we/ETA_final/DOC.csv') 

#### TOKEN table

In [27]:
# segmented into words
nltk.download('averaged_perceptron_tagger')
df = df.sent_str.apply(lambda x: pd.Series(nltk.pos_tag(nltk.word_tokenize(x))))
df = df.stack().to_frame().rename(columns = {0:'pos_tuple'})
df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/yk4we/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


pos_tuple
doc_id                          
48      0  0            (I, PRP)
           1            (’, VBP)
           2            (ve, RB)
           3         (been, VBN)
           4      (falling, VBG)
...                          ...
1026333 17 20  (represents, VBZ)
           21            (a, DT)
           22          (new, JJ)
           23          (low, JJ)
           24             (., .)

[5394914 rows x 1 columns]

In [28]:
df['pos'] = df.pos_tuple.apply(lambda x: x[1])
df['token_str'] = df.pos_tuple.apply(lambda x: x[0])

In [29]:
df.index.names = ['doc_id','sent_num','token_num']

In [30]:
df

pos_tuple  pos   token_str
doc_id  sent_num token_num                                    
48      0        0                   (I, PRP)  PRP           I
                 1                   (’, VBP)  VBP           ’
                 2                   (ve, RB)   RB          ve
                 3                (been, VBN)  VBN        been
                 4             (falling, VBG)  VBG     falling
...                                       ...  ...         ...
1026333 17       20         (represents, VBZ)  VBZ  represents
                 21                   (a, DT)   DT           a
                 22                 (new, JJ)   JJ         new
                 23                 (low, JJ)   JJ         low
                 24                    (., .)    .           .

[5394914 rows x 3 columns]

In [31]:
# TOKEN table
TOKEN = df.copy()
#remove non-alphabetic token (e.g.punctuation)
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')
#remove NA
TOKEN = TOKEN[~TOKEN.term_str.isna()]
TOKEN.term_str = TOKEN.term_str.replace('',np.NaN)
TOKEN = TOKEN[~TOKEN.term_str.isna()]
TOKEN

pos_tuple  pos   token_str    term_str
doc_id  sent_num token_num                                                
48      0        0                   (I, PRP)  PRP           I           i
                 2                   (ve, RB)   RB          ve          ve
                 3                (been, VBN)  VBN        been        been
                 4             (falling, VBG)  VBG     falling     falling
                 5               (behind, IN)   IN      behind      behind
...                                       ...  ...         ...         ...
1026333 17       19             (people, NNS)  NNS      people      people
                 20         (represents, VBZ)  VBZ  represents  represents
                 21                   (a, DT)   DT           a           a
                 22                 (new, JJ)   JJ         new         new
                 23                 (low, JJ)   JJ         low         low

[4623804 rows x 4 columns]

#### VOCAB table

In [32]:
#count the value for each term from TOKEN table
VOCAB = TOKEN.term_str.value_counts().to_frame()\
    .rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [33]:
# num = 1 if the term is number
VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')
VOCAB

,term_str,n,num
term_id,,,
0,0,28,1
1,00,6,1
2,000,4,1
3,000000,1,1
4,00000001,1,1
...,...,...,...
79232,گر,1,0
79233,گروههای,1,0
79234,گزارش,1,0


### Annotate(VOCAB)

In [34]:
nltk.download('stopwords')
#stopwords
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1
sw.sample(10)

[nltk_data] Downloading package stopwords to /home/yk4we/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,dummy
term_str,
few,1
into,1
s,1
re,1
over,1
an,1
up,1
this,1
their,1


In [35]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [36]:
VOCAB[VOCAB.stop == 1]

,term_str,n,num,stop
term_id,,,,
3733,a,96755,0,1
3948,about,15492,0,1
3958,above,726,0,1
4839,after,5013,0,1
4861,again,1780,0,1
...,...,...,...,...
78421,you,14507,0,1
78453,your,2852,0,1
78455,yours,25,0,1


#### Add Stems

In [37]:
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.term_str.apply(stemmer1.stem)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.term_str.apply(stemmer2.stem)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.term_str.apply(stemmer3.stem)

In [38]:
TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)
TOKEN.head()

pos_tuple  pos token_str term_str  term_id
doc_id sent_num token_num                                                 
48     0        0                (I, PRP)  PRP         I        i    34703
                2                (ve, RB)   RB        ve       ve    74875
                3             (been, VBN)  VBN      been     been     9494
                4          (falling, VBG)  VBG   falling  falling    26448
                5            (behind, IN)   IN    behind   behind     9579

## BOW, TFIDF and Vector Space

In [39]:
VOCAB['pos_max'] = TOKEN.groupby(['term_id', 'pos']).pos.count().unstack().idxmax(1)

In [40]:
VOCAB.sample(5)

,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster,pos_max
term_id,,,,,,,,
32442,hasidim,1,0,0,hasidim,hasidim,hasidim,NNP
7235,arbitrarilydefined,1,0,0,arbitrarilydefin,arbitrarilydefin,arbitrarilydefin,JJ
28429,fosia,16,0,0,fosia,fosia,fos,NNP
22986,duet,3,0,0,duet,duet,duet,NN
58258,reformists,1,0,0,reformist,reformist,reform,NNS


In [42]:
LIB.to_csv('/home/yk4we/ETA_final/LIB.csv')
DOC.to_csv('/home/yk4we/ETA_final/DOC.csv')
TOKEN.to_csv('/home/yk4we/ETA_final/TOKEN.csv')
VOCAB.to_csv('/home/yk4we/ETA_final/VOCAB.csv')

## TFIDF

In [46]:
TOKEN.reset_index()
VOCAB.sample(10)

,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster,pos_max
term_id,,,,,,,,
50035,oppositionresearch,8,0,0,oppositionresearch,oppositionresearch,oppositionresearch,JJ
55801,propitious,1,0,0,propiti,propiti,prop,JJ
35390,impressionist,3,0,0,impressionist,impressionist,impress,JJ
50559,outreach,50,0,0,outreach,outreach,outreach,NN
8981,bargaining,43,0,0,bargain,bargain,bargain,NN
15872,collège,1,0,0,collèg,collèg,collèg,NNP
6093,analyzing,22,0,0,analyz,analyz,analys,VBG
39676,ksue,1,0,0,ksue,ksue,ksu,NNP
12065,buckingham,7,0,0,buckingham,buckingham,buckingham,NNP


In [47]:
from sklearn 

SyntaxError: invalid syntax (<ipython-input-47-f2cfe0daf8e3>, line 1)

In [ ]:
def TFIDF(TOKEN, bag, tf_method, idf_method, count_method, count_item):
    TOKEN = TOKEN[~TOKEN.term_str.isna()]
    TOKEN['stem_porter'] = TOKEN.term_id.map(VOCAB.stem_porter)
    
    if count_item == 'term':
        print('term')
        BOW = TOKEN.groupby(bag+['term_id']).term_id.count().to_frame().rename(columns={'term_id':'n'})
        BOW['c'] = BOW.n.astype('bool').astype('int')
        
    else: #stem
        BOW = TOKEN.groupby(bag+['stem_porter']).stem_porter.count().to_frame().rename(columns={'stem_porter':'n'})
        BOW['c'] = BOW.n.astype('bool').astype('int')
    
    DTCM = BOW[count_method].unstack().fillna(0).astype('int')
    print('dtcm')
    if tf_method == 'sum':
        print('sum')
        TF = DTCM.T / DTCM.T.sum()

    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()

    elif tf_method == 'log':
        TF = np.log10(1 + DTCM.T)

    elif tf_method == 'raw':
        TF = DTCM.T

    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
        TF = 0.5 + (1 - 0.5) * TF[TF > 0] #switched tf_norm_k out as 0.5

    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
        
    print('done sum')
    TF = TF.T
    print('tf')

    DF = DTCM[DTCM > 0].count()
    print('idf')
    # compute IDF
    N = DTCM.shape[0]
    if idf_method == 'standard':
        print('standard')
        IDF = np.log10(N / DF)

    elif idf_method == 'max':
        IDF = np.log10(DF.max() / DF) 

    elif idf_method == 'smooth':
        IDF = np.log10((1 + N) / (1 + DF))

    print('multiply')
    TFIDF = TF * IDF
    return TFIDF, DF, IDF

In [48]:
OHCO = ['doc_id', 'sent_num', 'token_num']

BAG = OHCO[:1]
TFIDF, DF, IDF=TFIDF(TOKEN, BAG, 'sum', 'standard', 'n', 'term')
TFIDF.head()

term
dtcm
sum
done sum
tf
idf
standard
multiply


term_id,0,1,2,3,4,5,6,7,8,9,...,80420,80421,80422,80423,80424,80425,80426,80427,80428,80429
doc_id,,,,,,,,,,,,,,,,,,,,,
48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
TFIDF

NameError: name 'TFIDF' is not defined

In [50]:
TFIDF_books = pd.read_csv('/home/yk4we/ETA_final/TFIDF_BOOKS.csv')
TFIDF_books

,doc_id,0,1,2,3,4,5,6,7,8,...,80420,80421,80422,80423,80424,80425,80426,80427,80428,80429
0,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9767,1026329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9768,1026330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9769,1026331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9770,1026332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
TFIDF_books.sum()

doc_id    4.844867e+09
0         2.158094e-01
1         2.532588e-02
2         2.606081e-02
3         7.838867e-03
              ...     
80425     1.334443e-02
80426     1.334443e-02
80427     1.334443e-02
80428     1.334443e-02
80429     1.334443e-02
Length: 80429, dtype: float64

In [76]:
VOCAB['df'] = DF
VOCAB['idf'] = IDF
VOCAB['tfidf_sum'] = TFIDF.sum()
VOCAB.head()

,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster,pos_max,df,idf,tfidf_sum
term_id,,,,,,,,,,,
0,0,28,1,0,0,0,0,CD,24,2.609772,0.215809
1,00,6,1,0,00,00,00,JJ,5,3.291013,0.025326
2,000,4,1,0,000,000,000,CD,3,3.512862,0.026061
3,000000,1,1,0,000000,000000,000000,NN,1,3.989983,0.007839
4,00000001,1,1,0,00000001,00000001,00000001,CD,1,3.989983,0.004493


In [79]:
#top 20 words in the corpus by TFIDF sum (excluding stop words)
## words with bag of 'doc_id'
VOCAB.loc[VOCAB.stop == 0].sort_values('tfidf_sum', ascending=False).head(20).style.background_gradient(cmap='YlGn', high=1)

,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster,pos_max,df,idf,tfidf_sum
term_id,,,,,,,,,,,
73344,trump,22979,0,0,trump,trump,trump,NNP,4911,0.298814,15.458281
55796,president,11611,0,0,presid,presid,presid,NNP,4413,0.345250,8.816856
20547,democrats,6039,0,0,democrat,democrat,democr,NNPS,2557,0.582253,8.698520
50385,omar,2520,0,0,omar,omar,om,NNP,340,1.458505,8.335294
49711,obama,4820,0,0,obama,obama,obam,NNP,1895,0.712374,7.389082
27190,fbi,3303,0,0,fbi,fbi,fbi,NNP,912,1.029989,7.041603
15475,clinton,3578,0,0,clinton,clinton,clinton,NNP,1349,0.859972,6.929748
75579,us,8376,0,0,us,us,us,NNP,3975,0.390646,6.766351
55016,post,4468,0,0,post,post,post,NNP,2185,0.650532,6.710862


## Word Embedding: Word2Vec
create word embedding using word2vec and visualize result with tSNE

In [53]:
from gensim.models import word2vec
from sklearn.manifold import TSNE
import plotly_express as px

### Import TOKEN and convert to Corpus for Gensim
We import data from the TOKEN table of the novels corpus, excluding proper nouns.


In [54]:
LIB.sample(5)

,doc_title,year,doc_source
doc_id,,,
26781,Large impacts may have nurtured life on Earth,2016,Daily Kos
740721,Trump Wins!,2019,PowerLine
173996,Midday open thread: Right-wing radio biz model...,2017,Daily Kos
193602,New study finds Latinos have helped keep Ameri...,2017,Daily Kos
217591,Democrats propose working together on tax refo...,2017,Daily Kos


In [55]:
LIB = LIB.reset_index()

In [63]:
powerline = LIB.loc[LIB.doc_source.str.contains('PowerLine')].doc_id.to_list()
daily = LIB.loc[LIB.doc_source.str.contains('Daily')].doc_id.to_list()

In [64]:
TOKEN = TOKEN.reset_index()

In [73]:
TOKEN_powerline = TOKEN.loc[TOKEN.doc_id.isin(powerline)]
TOKEN_daily= TOKEN.loc[TOKEN.doc_id.isin(daily)]

In [89]:
TOKEN.loc[TOKEN.doc_id == 81509]

,doc_id,sent_num,token_num,pos_tuple,pos,token_str,term_str,term_id
503792,81509,0,0,"(nan, NN)",NN,nan,nan,46870


In [74]:
TOKEN_powerline['source'] = 'powerline'
TOKEN_daily['source'] = 'daily'

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [72]:
LIB.sample(5)

,doc_id,doc_title,year,doc_source
17867,570748,The Oslo follies,2018,PowerLine
5957,81509,Republican calls for Sessions to recuse himsel...,2017,Daily Kos
11797,212659,Trump administration threatens Alaska's energy...,2017,Daily Kos
2858,30932,Giant mega-berg may be calving off of Larsen I...,2016,Daily Kos
3005,32203,Cartoon: How Putin hacked 2016,2016,Daily Kos


In [78]:
TOKEN_powerline['doc'] = TOKEN_powerline.doc_id.map(LIB.set_index('doc_id').doc_title)
TOKEN_daily['doc'] = TOKEN_daily.doc_id.map(LIB.set_index('doc_id').doc_title)

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [80]:
TOKEN_powerline.head()

,doc_id,sent_num,token_num,pos_tuple,pos,token_str,term_str,term_id,source,doc
0,48,0,0,"(I, PRP)",PRP,I,i,34703,powerline,These Zins Are a Dusi
1,48,0,2,"(ve, RB)",RB,ve,ve,74875,powerline,These Zins Are a Dusi
2,48,0,3,"(been, VBN)",VBN,been,been,9494,powerline,These Zins Are a Dusi
3,48,0,4,"(falling, VBG)",VBG,falling,falling,26448,powerline,These Zins Are a Dusi
4,48,0,5,"(behind, IN)",IN,behind,behind,9579,powerline,These Zins Are a Dusi


In [96]:
data.loc[data.doc_source == 'PoliticoMagagine']

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url,year


### Scikit-learn

In [64]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
cv=CountVectorizer()

In [75]:
data[['doc_id','doc_content']].set_index('')

,doc_id,doc_content
0,48,I’ve been falling behind on many regular feat...
1,49,"At least I think it was the Onion. Someone, a..."
2,50,Obamacare was enacted on a foundational set o...
3,51,It’s unfair to compare and contrast Barack Ob...
4,52,Public Policy Polling ran a survey here in Mi...
...,...,...
13395,1026329,"I think it was Tom Bethell, back when he wrot..."
13396,1026330,The Trump Department of Justice has steadfast...
13397,1026331,No one in public life has forgotten what the ...
13398,1026332,Today the 2nd Circuit Court of Appeals ruled ...


In [70]:
word_count_vector=cv.fit_transform(DOC)

In [71]:
word_count_vector.shape

(1, 1)

In [68]:
print('1'

SyntaxError: unexpected EOF while parsing (<ipython-input-68-942d8a4b88f5>, line 1)